In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import axis as ax
from gsw import SA_from_SP, CT_from_t, rho, p_from_z, spiciness0
%matplotlib inline

In [3]:
cd /Users/alaimo/Documents/Rutgers/RoadToPhD/Chapter2

/Users/alaimo/Documents/Rutgers/RoadToPhD/Chapter2


In [48]:
nefsc_count = pd.read_csv("Data/22560_NEFSCFallFisheriesIndependentBottomTrawlData/22560_UNION_FSCS_SVCAT.csv", on_bad_lines="skip", encoding='unicode_escape').groupby('SVSPP').get_group(503)
nefsc_ocean = pd.read_csv("Data/22560_NEFSCFallFisheriesIndependentBottomTrawlData/22560_UNION_FSCS_SVSTA_updated.csv", on_bad_lines="skip", encoding='unicode_escape')

/var/folders/1m/h30kdd551mzgc1v42hlv1v9w0000gn/T/ipykernel_9749/3525079727.py:1: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  nefsc_count = pd.read_csv("Data/22560_NEFSCFallFisheriesIndependentBottomTrawlData/22560_UNION_FSCS_SVCAT.csv", on_bad_lines="skip", encoding='unicode_escape').groupby('SVSPP').get_group(503)


In [49]:
nefsc_count

,CRUISE6,CRUISE,STRATUM,TOW,STATION,STATUS_CODE,ID,SVSPP,CATCHSEX,EXPCATCHNUM,EXPCATCHWT,SCIENTIFIC_NAME
50,196307,6307,1260,3,3,10,196307012600030003,503,0,7.0,0.000,Loligo pealeii (longfin squid)
120,196307,6307,1400,1,8,10,196307014000010008,503,0,19.0,0.000,Loligo pealeii (longfin squid)
676,196307,6307,1360,7,54,10,196307013600070054,503,0,12.0,0.000,Loligo pealeii (longfin squid)
796,196307,6307,1320,1,65,10,196307013200010065,503,0,1.0,0.000,Loligo pealeii (longfin squid)
1274,196307,6307,1250,2,104,10,196307012500020104,503,0,NaN,0.000,Loligo pealeii (longfin squid)
...,...,...,...,...,...,...,...,...,...,...,...,...
338062,202404,202404,3610,3,808,10,202404036100030808,503,0,13.0,0.505,Loligo pealeii (longfin squid)
338090,202404,202404,1260,2,810,10,202404012600020810,503,0,1.0,0.056,Loligo pealeii (longfin squid)
338120,202404,202404,1260,4,812,10,202404012600040812,503,0,65.0,3.979,Loligo pealeii (longfin squid)
338146,202404,202404,1260,6,814,10,202404012600060814,503,0,4.0,0.172,Loligo pealeii (longfin squid)


In [50]:
nefsc_count = nefsc_count.drop(columns=["CRUISE6","CRUISE","SVSPP",'TOW','STATION','STRATUM','STATUS_CODE','SCIENTIFIC_NAME',"CATCHSEX"])
nefsc_count = nefsc_count.rename(columns = {"EXPCATCHNUM":"squid", "EXPCATCHWT":"weight"})
nefsc_count['squid1'] = np.where(nefsc_count['squid'] >= 1.0,1,0).astype(int)

In [51]:
nefsc_count

,ID,squid,weight,squid1
50,196307012600030003,7.0,0.000,1
120,196307014000010008,19.0,0.000,1
676,196307013600070054,12.0,0.000,1
796,196307013200010065,1.0,0.000,1
1274,196307012500020104,NaN,0.000,0
...,...,...,...,...
338062,202404036100030808,13.0,0.505,1
338090,202404012600020810,1.0,0.056,1
338120,202404012600040812,65.0,3.979,1
338146,202404012600060814,4.0,0.172,1


In [58]:
nefsc_full = pd.merge(nefsc_ocean,nefsc_count,on="ID", how='left').sort_index()

In [59]:
nefsc_full['year'] = nefsc_full['ID'].astype(str).str[:4].astype(int)

In [61]:
nefsc_full.year.unique()

array([1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973,
       1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984,
       1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
       2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 2021, 2022, 2023, 2024])

In [ ]:
# nefsc_full["START_DATE"] = pd.to_datetime(nefsc_full.START_DATE, format='%m/%d/%y %H:%M', errors='coerce').dt.strftime('%m/%d/%y')
# nefsc_full['DOY']= pd.to_datetime(nefsc_full.START_DATE, format='%m/%d/%y', errors='coerce').dt.dayofyear
# nefsc_full['year']= pd.to_datetime(nefsc_full.START_DATE, format='%m/%d/%y', errors='coerce').dt.year
# # nefsc_full["START_DATE"] = pd.to_datetime(nefsc_full.START_DATE, format='%m/%d/%y %H:%M', errors='coerce').dt.strftime('%m/%d/%y')

In [62]:
drops = np.arange(1963,2023)
df2 = nefsc_full.set_index('year')
nefsc = df2.drop(drops, errors='ignore')

In [65]:
nefsc = nefsc.reset_index()

In [67]:
nefsc["START_DATE"]= pd.to_datetime(nefsc.START_DATE, format='%m/%d/%y %H:%M', errors='coerce').dt.strftime('%m/%d/%y')

In [73]:
towdates = nefsc["START_DATE"].copy()

In [74]:
towdates

0      09/10/23
1      09/10/23
2      09/10/23
3      09/10/23
4      09/10/23
         ...   
754    11/10/24
755    11/10/24
756    11/11/24
757    11/11/24
758    11/11/24
Name: START_DATE, Length: 759, dtype: object

In [75]:
towdates['region'] = 'adult_northeast'

In [79]:
towdates = towdates.drop_duplicates()

In [80]:
towdates.to_csv("Data/towcatchdates_fall.csv")